In [1]:
import numpy as np
import torch
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from arch import arch_model
from pytorch_lightning.loggers import  TensorBoardLogger
from scipy.optimize import root

In [2]:

# Генерация данных
def generate_ground_garch(omega, d, phi, beta, n=3000):
    am = arch_model(None, mean='Zero', vol='FIGARCH', p=1, q=1, power=2)
    params = np.array([omega, d, phi, beta])
    am_data = am.simulate(params, n)
    return am_data['data'].to_numpy(), am_data['volatility'].to_numpy()


In [3]:

# Dataset для полных последовательностей
class FullSequenceDataset(Dataset):
    def __init__(self, eps, vol, truncation_size, scale=100):
        super().__init__()
        self.eps_squared = torch.tensor(np.square(eps) * scale)
        self.vol = (torch.tensor(vol)- 0.095) * scale
        self.truncation_size = truncation_size
        self.eps_squared = self.eps_squared.float()
        self.vol = self.vol.float()

    def __len__(self):
        return len(self.eps_squared) - self.truncation_size  # Количество возможных окон

    def __getitem__(self, idx):
        # Возвращаем окно остатков и соответствующий таргет
        return (
            self.eps_squared[idx:idx+self.truncation_size],
            self.eps_squared[idx+self.truncation_size]
        )


In [4]:
class FIGARCHDM(pl.LightningDataModule):
    def __init__(self, eps, vol, truncation_size, batch_size):
        super().__init__()

        self.eps = eps
        self.vol = vol
        self.truncation_size = truncation_size
        self.batch_size = batch_size
    def setup(self, stage = None):
        self.train_dataset = FullSequenceDataset(self.eps, self.vol, self.truncation_size)


    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size= self.batch_size ,shuffle=False)

In [5]:
class CorrectedNLoss(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, pred_var, target_eps_squared):

        loss =  (0.5*(torch.log(pred_var)) + (target_eps_squared/(2*pred_var))) 
        return loss.mean()

In [6]:

# Модель FIGARCH с CNN
class FullConvFIGARCH(pl.LightningModule):
    def __init__(self, truncation_size, lr=1e-3):
        super().__init__()
        self.save_hyperparameters()
        self.truncation_size = truncation_size
        self.lr = lr
        
        self.weights = nn.Parameter(torch.linspace(1.0, 0.1, truncation_size))
        
        self.loss_fn = CorrectedNLoss()

    def forward(self, x):

        return torch.sum(x * self.weights, dim=1)

    def training_step(self, batch, batch_idx):
        eps_window, target_eps = batch
        pred_var = self.forward(eps_window)
        loss = self.loss_fn(pred_var, target_eps)
        self.log('train_loss', loss, prog_bar=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, 
            mode='min', 
            factor=0.5, 
            patience=3, 
            verbose=True
        )
        return {
            'optimizer': optimizer,
            'lr_scheduler': {
                'scheduler': scheduler,
                'monitor': 'train_loss'
            }
        }



In [7]:

omega, d, phi, beta = 0.1, 0.5, 0.2, 0.5
data, volat = generate_ground_garch(omega, d, phi, beta)


In [8]:
truncation_size = 10
batch_size = len(data) - truncation_size
 

In [9]:
model = FullConvFIGARCH(truncation_size)
dm = FIGARCHDM(data, volat, truncation_size, batch_size)


In [10]:
logger = TensorBoardLogger('tb_logs', 'figarch_model')

In [11]:
trainer = pl.Trainer(max_epochs=1200, accelerator='auto', logger= logger)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [12]:
trainer.fit(model, dm)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(

  | Name         | Type           | Params | Mode 
--------------------------------------------------------
0 | loss_fn      | CorrectedNLoss | 0      | train
  | other params | n/a            | 10     | n/a  
--------------------------------------------------------
10        Trainable params
0         Non-trainable params
10        Total params
0.000     Total estimated model params size (MB)
1         Modules in train mode
0         Modules in eval mode
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `D

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=1200` reached.


In [13]:
print("Learned weights:", model.weights.data)

Learned weights: tensor([0.1965, 0.1042, 0.1110, 0.0868, 0.1048, 0.1119, 0.0760, 0.1181, 0.1075,
        0.2992])


In [14]:
def decompute_lambdas(lambdas):
    truncation_size = len(lambdas)
    
    def equations(vars):
        d, phi, beta = vars
        eq1 = phi - beta + d - lambdas[0]
        eq2 = (d - beta) * (beta - phi) + d * (1 - d) / 2 - lambdas[1]

        term1 = beta * (d * beta - d * phi - beta**2 + beta * phi + d * (1 - d) / 2)
        term2 = d * (1 - d) / 2 * ((2 - d) / 3 - phi)
        eq3 = term1 + term2 - lambdas[2]

        return [eq1, eq2, eq3]



    initial_guess = [0.1, 0.1, 0.1] 
    sol = root(equations, initial_guess)

    return tuple(sol.x)

In [ ]:
decompute_lambdas(model.weights.data)

#Думаю нужно разобраться с выходами модели - возможно несоответствие по индексам

(0.7830661746919022, 0.1531962604312119, 0.7404699802776804)